# Mozart's Musikalisches Würfelspiel, K.516f

<img src="figs/wuerfelspiel_anleitung.png" alt="Würfelspiel Instructions" width=400> <img src="figs/mozart_wuerfelspiel.webp" alt="Würfelspiel Instructions" width=300>


> "To compose without the least knowledge of Music so much German Walzer or Schleifer as one pleases, by throwing a certain number with two dice."

In this notebook we will explore one of the early classics of algorithmic composition, Mozart's musical dice game.




In [ ]:
import numpy as np
from typing import List, Union, Dict, Tuple
import partitura as pt
import os
from partitura.utils.fluidsynth import HAS_FLUIDSYNTH
import pandas as pd

if HAS_FLUIDSYNTH:
    from partitura.utils.fluidsynth import (
        synthesize_fluidsynth as synthesize,
        SAMPLE_RATE,
    )
else:
    from partitura.utils.synth import synthesize, SAMPLE_RATE
import IPython.display as ipd

from wuerfelspiel_helper import DICE_THROWING_TABLE, pretty_print_dice_throwing_table

RNG = np.random.RandomState(1984)

<!-- ![wuerfelspiel_table](figs/wuefelspiel_table_p1.png) -->
<img src="figs/wuefelspiel_table_p1.png" alt="Description" width="400"> <img src="figs/wuefelspiel_table_p2.png" alt="Description" width="400">

<img src="figs/wuefelspiel_score_snippet.png" alt="Description" width="800">


## Basic Instructions

1. The letters A-H on each column specify each measure in the piece (the first table is for the first part and the second table for the second part of the piece).
2. For each measure, we roll 2 6-sided dice, we will get a result between 2 and 12.
3. We look up at the corresponding row on the table, which will tell us the number of the measure to use.

Here is a more readable version of the table:

In [ ]:
pretty_print_dice_throwing_table()

## Implmenenting the game!

What we need to implement:

1. A function that simulates rolling the dice
2. A way of generating a trajectory of dice roll for each measure
3. Load the score and split it into measures
4. Put the piece together using the generated trajectory.

In [ ]:
def roll_dice(rng: Union[np.random.RandomState, int] = RNG) -> int:
    """
    Simulates the roll of two 6-sided dice and returns their sum.

    Returns
    -------
    int
        The sum of two 6-sided dice rolls.
    """
    if isinstance(rng, int):
        rng = np.random.RandomState(rng)

    return int(rng.randint(1, 6) + rng.randint(1, 6))



In [ ]:

def generate_trajectory(rng: Union[np.random.RandomState, int] = RNG) -> np.ndarray:
    """
    Simulates rolling two dice 16 times and retrieves values from the table using
    the dice sum as the row index for each column.

    Returns
    -------
    np.ndarray
        A 1D numpy array representing the trajectory of values based on the dice rolls.
    """
    trajectory = []
    for col in range(16):
        dice_sum = roll_dice(rng)
        # Ensure the dice sum is between 2 and 12 (valid row indices)
        row_index = dice_sum - 2
        value = DICE_THROWING_TABLE[row_index, col]
        trajectory.append(value)

    return np.array(trajectory, dtype=int)

Now that we have a way to generate samples, we load the score and extract the notes for each measure pattern.

In [ ]:
# We load the score
score = pt.load_musicxml(
    os.path.join(
        "example_data",
        "Musikalisches_Wuerfelspiel_K.516f.musicxml",
    ),
    force_note_ids=True,
)
part = score[0]

bm = part.beat_map
note_array = part.note_array()
onsets = note_array["onset_beat"]
measures = dict()

# Get the notes for each measure in the score
for i, measure in enumerate(part.iter_all(pt.score.Measure)):
    # start and end time of the measure in divs
    measure_start_div = measure.start.t
    measure_end_div = measure.end.t

    # start and end time of the measure in beats
    measure_start_beat = bm(measure_start_div)
    measure_end_beat = bm(measure_end_div)

    # duration of the measure
    measure_duration = measure_end_beat - measure_start_beat

    # Get the the notes in the measure
    note_indices = np.where(
        np.logical_and(onsets >= measure_start_beat, onsets < measure_end_beat)
    )[0]

    measure_notes = note_array[note_indices]

    # Adjust the onset of the notes to start at the beginning of the measure
    measure_notes["onset_beat"] -= measure_start_beat
    measure_notes["onset_div"] -= measure_start_div

    # add the measures to the dictionary
    measures[i + 1] = (measure_notes, measure_duration)

Now we can put everything together into a function to generate a new Walzer!

In [ ]:
def compose_piece(
    measures: Dict[int, Tuple[np.ndarray, float]] = measures,
    rng: Union[np.random.RandomState, int] = RNG,
) -> np.ndarray:
    """Compose a piece!

    Parameters
    ----------
    measures : Dict[int, Tuple[np.ndarray, float]], optional
        Dictionary of measures from the piece
    rng : Union[np.random.RandomState, int], optional
        random number generator

    Returns
    -------
    generated_piece: np.ndarray
        The note array of the generated piece
    """

    # Generate a trajectory
    trajectory = generate_trajectory(rng)

    generated_piece = []
    starting_onset = 0
    # for each dice throw in the trajectory
    # get the notes and adjust their initial onset time
    for i, t in enumerate(trajectory):
        measure, measure_duration = measures[t]
        out_measure = measure.copy()
        out_measure["onset_beat"] += starting_onset
        starting_onset += measure_duration
        generated_piece.append(out_measure)

    generated_piece = np.concatenate(generated_piece)

    return generated_piece

Finally, let's hear the output!

In [ ]:
# Generate the piece
generated_piece = compose_piece(rng=2024)

# Synthesize audio
output = synthesize(note_info=generated_piece, bpm=100)

ipd.display(ipd.Audio(data=output, rate=SAMPLE_RATE, normalize=True))